(sec:setup-git)=
# Git環境の設定

Gitとは、プログラミングのソースコード等の更新履歴を管理するためのバージョン管理システム (VCS, Version Cotrol System)の一種です。バージョン管理システムにはGit以外にもSubversionやMercurialなどがある。

Gitを利用するには、
1. [GitHub](https://github.com/)や[BitBucket](https://bitbucket.org)等のウェブサービスを使う
2. 自分でGitサーバーを立てて使う

の2通りの方法が考えらるが、よほど機密性の高いソースコードを扱う等の場合でなければ、ウェブサービスを使う方法で問題ない。

本節では、GitHubを用いてGitレポジトリの作成とソースコードの管理を行う方法を紹介する。

## Gitについて

Gitは**分散型バージョン管理システム**であり、一つのプロジェクトに関連するコードを複数のユーザが別々に編集することを想定して設計されている。

Gitにおいては、ここのユーザが作成した変更が**コミット**という形で管理され、異なるユーザが作成したコミットを**マージ**することにより、コードを更新していく。

一方、Git以前に主流であったSubversionは**中央集権的なバージョン管理システム**であり、とあるユーザがソースコードを編集している間は、サーバ上のソースコードの更新がロックされるという仕組みであった。そのため、複数の異なるユーザが同時にソースコードを編集するようなプロジェクトには不向きであった。

現在、多くのソフトウェアプロジェクトは、システムの大規模化に伴い、複数のユーザで共同開発されるのが一般的になっており、Gitは現在のプログラミング・パラダイムにより即した仕組みであると言える。

```{admonition} Gitの特長
:class: note

- 分散型であるために複数のユーザが同時にコードを編集可能
- 各ユーザの編集内容はローカル環境で「コミット」という単位で管理される
- 複数のユーザが行った編集が「マージ」されることでコードが更新されていく
```

## GitHubアカウントの作成

公式アナウンスに従ってアカウントを作成しよう。ユーザ名は変更できないので慎重に考えること。

<https://docs.github.com/ja/get-started/quickstart/creating-an-account-on-github>

## SSHキーの登録

2024年現在、GitHubのプライベート・リポジトリ上でソースコードを管理するためにはSSH (Secure Shellの略)を使用する必要がある。

Gitと同様、Mac/Linuxでは特別なインストールは必要なく、ターミナル上で`ssh --version`とタイプしてEnterを押すと、SSHのバージョンが表示される。

一方、Windowsの場合には、SSHの機能をOS付属のオプションから有効化する必要がある。詳細はMicrosoftの公式ガイドを参考にしてほしい。
<https://learn.microsoft.com/ja-jp/windows-server/administration/openssh/openssh_install_firstuse?tabs=gui>

SSHの用意ができたら、`ssh-keygen`コマンドを使ってSSHキーを生成する。SSHキーにはいくつかの種類があるが、今回はRSA暗号を用いた4096ビット長のキーを用いる。

```shell
ssh-keygen -r rsa -b 4096
```

すると、パスワードの入力を求められるが、今回はパスワードは設定しなくて良い (本当は設定した方が良いが、よく分からない場合は設定すると混乱する)。

ただしくSSHキーの生成が完了すると、ホームディレクトリ直下に`.ssh`という名前のフォルダが作成され、そこに`id_rsa`と`id_rsa.pub`という2つのファイルが生成される。なお、`.ssh`フォルダはMacやLinuxでは隠しフォルダ扱いで、単に`ls`とするだけではリストに現れないので注意すること (代わりに`ls -a`を使う)。

この`id_rsa`と`id_rsa.pub`は前者が秘密鍵、後者が公開鍵と呼ばれる。公開鍵はその名前の通り、他の人が見られる場所に置いても良く、こちらがGitHubに登録する方のファイルになる。一方で、秘密鍵はその名前の通り、秘密の鍵なので、これは他の人に共有したりしないように注意しよう。

公開鍵`id_rsa.pub`をテキストエディタで開くか、`cat`コマンドで内容を表示し、その内容をコピーする。

具体的には、[GitHubの設定ページ](https://github.com/settings)の左にあるメニューから「SSH and GPG keys」を選ぶ (下図参照)。

```{image} ./imgs/SSH_and_GPG_key.jpg
:align: center
:width: 40%
:class: with-border
```

現れる画面で「New SSH Key」ボタンを押す。すると、SSHキーの公開鍵の内容を入力するテキストボックスが現れるので、ここに先ほどコピーした公開鍵の内容をペーストする (下図参照)。

```{image} ./imgs/new_SSH_key.jpg
:align: center
:width: 100%
:class: with-border
```

ここまで完了したらローカル環境に戻り、シェルを開く。その上で、秘密鍵である`id_rsa`のパーミッション (ファイルに対してどんな操作ができるかを制限する仕組み)を設定する。具体的には、自分以外のユーザは何もできず、自分も読み取りしかできないように設定する。

```shell
chmod 600 ~/.ssh/id_rsa
```

これでSSHキーの登録は完了なので、ローカル環境でシェルを開き、

```shell
ssh -i ~/.ssh/id_rsa git@github.com
```

と、コマンドを打ってEnterを押す。もし、SSH接続に成功すれば以下のメッセージが現れる。

> Hi xxxx! You've successfully authenticated, but GitHub does not provide shell access.
> Connection to github.com closed.

もし、上記のようにSSH接続が上手くいかない場合には、GitHubへの公開鍵の登録に失敗している可能性があるので、もう一度内容をよく確認すること。

## ローカル環境の準備

### Windows

Windowsは、初期状態ではGit環境が用意されていないので、別途ソフトウェアをインストールする必要がある。

最も一般的なものは[Git for Windows](https://gitforwindows.org/)なのだが、こちらを使う場合には、Gitコマンドに環境変数のパスを通すなど、余計な設定が必要になるため、インストールの際に注意が必要。

環境変数のパスを通す設定をし損ねた場合には、自分で環境変数のパスを通すか、(慣れないと使い勝手が悪い)Git Bashというシステムを使うことになる。

そこでおすすめしたいのが、PowerShell上で動く、ソフトウェア管理ツールである[Chocolatey](https://chocolatey.org/)である。Chocolateyをインストールするための(おすすめ)手順は以下の通り。

1. オープンソース版の[PowerShell](https://github.com/PowerShell/PowerShell/releases/tag/v7.4.1)をインストールする。

2. PowerShellを**管理者権限で**開き、[Install Chocolatey](https://chocolatey.org/install#individual)のページ中段にある以下のコマンドを実行する。

```powershell
Set-ExecutionPolicy Bypass -Scope Process -Force; [System.Net.ServicePointManager]::SecurityProtocol = [System.Net.ServicePointManager]::SecurityProtocol -bor 3072; iex ((New-Object System.Net.WebClient).DownloadString('https://community.chocolatey.org/install.ps1'))
```

3. PowerShellを開き、`choco list -l`とタイプしてEnterを押す。正しくコマンドが認識されていれば、現在のChocolateyのバージョンが表示される。

ここまででChocolateyのインストールは完了なので、最後にChocolateyを使ってGitをインストールする ([参考ページ](https://community.chocolatey.org/packages/git))。

```powershell
choco install git
```

インストール完了後、PowerShell上で`git --version`とタイプしてEnterを押す。正しくコマンドが認識されていれば、現在のGitのバージョンが表示される。

### Mac / Linux

特に準備をする必要はなく、ターミナルから`git`コマンドが使用できる。念のため、ターミナルを開き `git --version`とタイプしてEnterを押す。正しくコマンドが認識されていれば、現在のGitのバージョンが表示される。

### GUIを通してGitを使う (非推奨)

もしGUIを用いて、視覚的にバージョン管理を行いたい場合は、[GitHub Desktop](https://desktop.github.com/)や[Sourcetree](https://www.sourcetreeapp.com/), [GitKraken](https://www.gitkraken.com/)などが比較的使いやすい。

しかし、GUIを使っていると、なかなか融通が利かない部分もあり、またGitのコマンドをなかなか覚えないので、基本はターミナルからコマンドを実行することを強く推奨する。

```{admonition} Gitのインストール
:class: note

- WindowsはGit for Windowsをインストールするか、Chocolateyを使ってGitをインストール
- MacやLinuxの場合には特段、インストールをしなくてもGitがインストールされている
- Gitをローカル環境で操作する場合にはシェル環境 (PowerShellやターミナル等)を使うことを推奨
```

## ローカル環境におけるGitリポジトリの作成

ローカル環境でGitリポジトリを作るには、まず、ソースコードを保存するディレクトリを作成する。ここではディレクトリ名が`my-first-repo`であるとして説明を進める。

ディレクトリの作成には`mkdir`コマンドを用いる。

```shell
# ディレクトリの作成
mkdir my-first-repo
```

その後、ディレクトリ内に移動し、そこでGitリポジトリの初期化を行う。

```shell
# ディレクトリを移動
cd my-first-repo
# Gitリポジトリの作成
git init
```

すると、ディレクトリ内に隠しフォルダとして`.git`というディレクトリが作られるので`ls -a`コマンドで、ディレクトリの存在を確認しておこう。

## コミットとプッシュ

Gitでまず覚えるべきコマンドは**コミット** (`git commit`)と**プッシュ**がある。まずは、このそれぞれについて、操作の内容とコマンドについて見ていこう。

**コミット**はローカル環境で行ったソースコードの更新をローカル環境の上に記録する操作を指す。この際、コミットは何回にも分けることができ、コミットを行うごとに、その時のソースコードの状態 (より具体的に言えば、直前のコミットからの差分情報) が保存される。

## Gitサーバ上でのリポジトリの作成

GitHubを用いてバージョン管理を開始するには**リポジトリ**を作成する必要がある。

本来、リポジトリを作成するには、Gitサーバ上で`git init --bare`というコマンドを実行する必要があるのだが、GitHubを用いる場合には、ウェブ上の操作によりGitリポジトリを作成することができる。

まずは、各自のGitHubのトップページ (`https://github.com/[ユーザ名]`)にアクセスし、右上の「＋」ボタンから「New Repository」を選ぶ (下図参照)。

```{image} ./imgs/create_repo_01.jpg
:align: center
:width: 60%
:class: with-border
```

すると、以下のような画面が現れるので、まずは適当にリポジトリ名 (Repository name)を決める。なお、リポジトリ名には英数字しか使うことができないので注意すること。リポジトリの説明文 (Description)は後からでも容易に変更できるので、現時点では特に入力する必要はない。

次に、画面を少し下にスクロールすると、リポジトリの可視性と初期化オプションを決定する項目がある (下図参照)。

```{image} ./imgs/create_repo_02.jpg
:align: center
:width: 60%
:class: with-border
```

リポジトリの可視性は、誰からでもアクセス可能な「Public」と、自分と指定したユーザのみがアクセス可能な「Private」がある。特にこだわりがなければ最初はPrivateにしておいて、ある程度、プログラムの開発が進んだ時点で「Public」にすると良いだろう。

最後のリポジトリの初期化オプションについては、リポジトリの説明を記述する`README.md`、無視するファイルを設定する`.gitignore`、そしてオープンソース・プロジェクトのライセンスをしていするか否か、を指定できる。それぞれの意味を理解しているなら指定しても構わないが、後からでも変更できるので、取り急ぎ指定する必要はない。

以上の設定を終了後、「Create repository」のボタンを押すと、リポジトリがGitHub上に作成される。

## ソースコードの編集・管理